# 인코더LSTM과 디코더LSTM으로 스마트 번역기 만들기


In [1]:
# 1. 패키지 수입
import numpy as np
import pandas as pd
from time import time

from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

# 하이퍼 파라미터 (정확도나 속도에 영향을 미치는 값)
MY_HIDDEN = 128
MY_EPOCH = 500

In [2]:
# 2. 번역 데이터 불러오기
raw = pd.read_csv('data/translate.csv', header=None)
eng_kor = raw.values.tolist() # 데이터 프레임을 list로 변환
print(len(eng_kor))
print(eng_kor[:3])

110
[['cold', '감기'], ['come', '오다'], ['cook', '요리']]


In [3]:
# 3. 영어알파벳과 한글알파벳 리스트 만들기

e_alpha=[c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
# {c:i for i,c in enumerate(e_alpha)}
k_alpah = pd.read_csv('data/korean.csv', header=None)[0].tolist()
alpha = e_alpha+k_alpah
char_to_num = {c:i for i,c in enumerate(alpha)}
alpha_total_size = len(char_to_num)
alpha_total_size

171

In [4]:
char_to_num.values()

dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170])

In [5]:
# 4. 문자당 num을 갖는 dict 만들기
print([char_to_num[eng] for eng in eng_kor[0][0]])
print([char_to_num[kor] for kor in eng_kor[0][1]])

[5, 17, 14, 6]
[32, 46]


In [6]:
to_categorical([char_to_num[eng] for eng in eng_kor[0][0]], num_classes=alpha_total_size)

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.,

In [7]:
np.eye(alpha_total_size)[[char_to_num[eng] for eng in eng_kor[0][0]]]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.,

In [8]:
# 5. 인코더 입력 (영어알파벳을 숫자로 바꾼 값을 원핫인코딩)
#    디코더 입력 ('S한글'을 숫자로 바꾼 값을 원핫인코딩)
#    디코더 출력 ('한글E'를 숫자로 바꾼값)
def encoding(eng_kor=eng_kor):
    enc_in=[]
    dec_in=[]
    dec_out=[]
    for row in eng_kor:
        eng = [char_to_num[c] for c in row[0]]
        kor = [char_to_num[c] for c in row[1]]
        # 인코더 입력
        enc_in.append(np.eye(alpha_total_size)[eng])
        # 디코더 입력
        dec_in.append(np.eye(alpha_total_size)[[char_to_num['S']]+kor])
        # 디코더 출력
        dec_out.append(kor+[char_to_num['E']])
    return enc_in, dec_in, dec_out

In [9]:
sample = [['wood', '나무'], ['word', '단어']]
encoding(sample)

([array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [10]:
# 6. 전체 번역 데이터 처리
X_enc, X_dec, Y_dec = encoding()

# numpy로 전환
X_enc = np.array(X_enc)
X_dec = np.array(X_dec)
Y_dec = np.array(Y_dec)
X_enc.shape, X_dec.shape, Y_dec.shape

((110, 4, 171), (110, 3, 171), (110, 3))

In [11]:
# 7. 모델 구현
# 인코더 LSTM 구현
ENC_IN = Input(shape=(4, alpha_total_size))
# 출력, state_c, state_h
_, state_h, state_c = LSTM(units=MY_HIDDEN, # 128
                          return_state=True)(ENC_IN)
# 인코더와 디코더 연결
link = [state_h, state_c]

# 디코더 구현
DEC_IN = Input(shape=(3, alpha_total_size))
DEC_MID = LSTM(units=MY_HIDDEN,
              return_sequences=True)(DEC_IN, initial_state=link)
# 최종 출력층
DEC_OUT = Dense(units=alpha_total_size,
               activation='softmax')(DEC_MID)

# 모델 구조 구현
model = Model(inputs=[ENC_IN, DEC_IN],
             outputs=DEC_OUT)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 4, 171)]     0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 3, 171)]     0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 128),        153600      ['input_1[0][0]']                
                                 (None, 128),                                                     
                                 (None, 128)]                                                     
                                                                                              

In [12]:
# 8. 학습과정 설정 & 학습하기
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='rmsprop',
             metrics=['accuracy'])
begin = time()
model.fit([X_enc, X_dec],
          Y_dec,
          epochs=MY_EPOCH,
          verbose=1)
end = time()
print('학습시간 :', end-begin)

Epoch 1/500
4/4 [==============================] - 4s 9ms/step - loss: 5.1085 - accuracy: 0.2273 
Epoch 2/500
4/4 [==============================] - 0s 10ms/step - loss: 4.9100 - accuracy: 0.3333
Epoch 3/500
4/4 [==============================] - 0s 9ms/step - loss: 3.9609 - accuracy: 0.3333
Epoch 4/500
4/4 [==============================] - 0s 8ms/step - loss: 3.4573 - accuracy: 0.3333
Epoch 5/500
4/4 [==============================] - 0s 8ms/step - loss: 3.4048 - accuracy: 0.3333
Epoch 6/500
4/4 [==============================] - 0s 7ms/step - loss: 3.3632 - accuracy: 0.3333
Epoch 7/500
4/4 [==============================] - 0s 7ms/step - loss: 3.3300 - accuracy: 0.3333
Epoch 8/500
4/4 [==============================] - 0s 7ms/step - loss: 3.3064 - accuracy: 0.3333
Epoch 9/500
4/4 [==============================] - 0s 7ms/step - loss: 3.2754 - accuracy: 0.3333
Epoch 10/500
4/4 [==============================] - 0s 8ms/step - loss: 3.2591 - accuracy: 0.3364
Epoch 11/500
4/4 [=========

4/4 [==============================] - 0s 8ms/step - loss: 1.1695 - accuracy: 0.8697
Epoch 85/500
4/4 [==============================] - 0s 10ms/step - loss: 1.1319 - accuracy: 0.8788
Epoch 86/500
4/4 [==============================] - 0s 7ms/step - loss: 1.0853 - accuracy: 0.8879
Epoch 87/500
4/4 [==============================] - 0s 7ms/step - loss: 1.0527 - accuracy: 0.8970
Epoch 88/500
4/4 [==============================] - 0s 7ms/step - loss: 1.0372 - accuracy: 0.8848
Epoch 89/500
4/4 [==============================] - 0s 7ms/step - loss: 1.0064 - accuracy: 0.9061
Epoch 90/500
4/4 [==============================] - 0s 8ms/step - loss: 0.9577 - accuracy: 0.9091
Epoch 91/500
4/4 [==============================] - 0s 10ms/step - loss: 0.9317 - accuracy: 0.9152
Epoch 92/500
4/4 [==============================] - 0s 8ms/step - loss: 0.8967 - accuracy: 0.9091
Epoch 93/500
4/4 [==============================] - 0s 8ms/step - loss: 0.8671 - accuracy: 0.9182
Epoch 94/500
4/4 [=============

Epoch 167/500
4/4 [==============================] - 0s 7ms/step - loss: 0.0303 - accuracy: 1.0000
Epoch 168/500
4/4 [==============================] - 0s 7ms/step - loss: 0.0305 - accuracy: 1.0000
Epoch 169/500
4/4 [==============================] - 0s 8ms/step - loss: 0.0280 - accuracy: 1.0000
Epoch 170/500
4/4 [==============================] - 0s 8ms/step - loss: 0.0280 - accuracy: 1.0000
Epoch 171/500
4/4 [==============================] - 0s 7ms/step - loss: 0.0299 - accuracy: 0.9970
Epoch 172/500
4/4 [==============================] - 0s 8ms/step - loss: 0.0232 - accuracy: 1.0000
Epoch 173/500
4/4 [==============================] - 0s 5ms/step - loss: 0.0233 - accuracy: 1.0000
Epoch 174/500
4/4 [==============================] - 0s 7ms/step - loss: 0.0208 - accuracy: 1.0000
Epoch 175/500
4/4 [==============================] - 0s 8ms/step - loss: 0.0202 - accuracy: 1.0000
Epoch 176/500
4/4 [==============================] - 0s 8ms/step - loss: 0.0242 - accuracy: 1.0000
Epoch 177/

4/4 [==============================] - 0s 7ms/step - loss: 3.4909e-04 - accuracy: 1.0000
Epoch 249/500
4/4 [==============================] - 0s 8ms/step - loss: 3.6768e-04 - accuracy: 1.0000
Epoch 250/500
4/4 [==============================] - 0s 8ms/step - loss: 0.0025 - accuracy: 1.0000
Epoch 251/500
4/4 [==============================] - 0s 8ms/step - loss: 2.9149e-04 - accuracy: 1.0000
Epoch 252/500
4/4 [==============================] - 0s 7ms/step - loss: 2.3013e-04 - accuracy: 1.0000
Epoch 253/500
4/4 [==============================] - 0s 8ms/step - loss: 2.1435e-04 - accuracy: 1.0000
Epoch 254/500
4/4 [==============================] - 0s 8ms/step - loss: 2.0067e-04 - accuracy: 1.0000
Epoch 255/500
4/4 [==============================] - 0s 8ms/step - loss: 1.8956e-04 - accuracy: 1.0000
Epoch 256/500
4/4 [==============================] - 0s 8ms/step - loss: 1.7729e-04 - accuracy: 1.0000
Epoch 257/500
4/4 [==============================] - 0s 8ms/step - loss: 1.6730e-04 - accur

4/4 [==============================] - 0s 7ms/step - loss: 6.0179e-06 - accuracy: 1.0000
Epoch 328/500
4/4 [==============================] - 0s 8ms/step - loss: 5.9171e-06 - accuracy: 1.0000
Epoch 329/500
4/4 [==============================] - 0s 7ms/step - loss: 5.5757e-06 - accuracy: 1.0000
Epoch 330/500
4/4 [==============================] - 0s 8ms/step - loss: 5.4569e-06 - accuracy: 1.0000
Epoch 331/500
4/4 [==============================] - 0s 7ms/step - loss: 5.3109e-06 - accuracy: 1.0000
Epoch 332/500
4/4 [==============================] - 0s 7ms/step - loss: 5.3080e-06 - accuracy: 1.0000
Epoch 333/500
4/4 [==============================] - 0s 7ms/step - loss: 5.1650e-06 - accuracy: 1.0000
Epoch 334/500
4/4 [==============================] - 0s 8ms/step - loss: 4.7799e-06 - accuracy: 1.0000
Epoch 335/500
4/4 [==============================] - 0s 8ms/step - loss: 4.5928e-06 - accuracy: 1.0000
Epoch 336/500
4/4 [==============================] - 0s 8ms/step - loss: 4.4888e-06 - a

4/4 [==============================] - 0s 8ms/step - loss: 1.2152e-06 - accuracy: 1.0000
Epoch 407/500
4/4 [==============================] - 0s 10ms/step - loss: 1.2044e-06 - accuracy: 1.0000
Epoch 408/500
4/4 [==============================] - 0s 8ms/step - loss: 1.1975e-06 - accuracy: 1.0000
Epoch 409/500
4/4 [==============================] - 0s 8ms/step - loss: 1.1748e-06 - accuracy: 1.0000
Epoch 410/500
4/4 [==============================] - 0s 8ms/step - loss: 1.1668e-06 - accuracy: 1.0000
Epoch 411/500
4/4 [==============================] - 0s 8ms/step - loss: 1.1596e-06 - accuracy: 1.0000
Epoch 412/500
4/4 [==============================] - 0s 8ms/step - loss: 1.1509e-06 - accuracy: 1.0000
Epoch 413/500
4/4 [==============================] - 0s 8ms/step - loss: 1.1375e-06 - accuracy: 1.0000
Epoch 414/500
4/4 [==============================] - 0s 8ms/step - loss: 1.1184e-06 - accuracy: 1.0000
Epoch 415/500
4/4 [==============================] - 0s 8ms/step - loss: 1.1014e-06 - 

4/4 [==============================] - 0s 7ms/step - loss: 6.3325e-07 - accuracy: 1.0000
Epoch 486/500
4/4 [==============================] - 0s 8ms/step - loss: 6.2856e-07 - accuracy: 1.0000
Epoch 487/500
4/4 [==============================] - 0s 8ms/step - loss: 6.2458e-07 - accuracy: 1.0000
Epoch 488/500
4/4 [==============================] - 0s 8ms/step - loss: 6.1917e-07 - accuracy: 1.0000
Epoch 489/500
4/4 [==============================] - 0s 7ms/step - loss: 6.1483e-07 - accuracy: 1.0000
Epoch 490/500
4/4 [==============================] - 0s 8ms/step - loss: 6.0941e-07 - accuracy: 1.0000
Epoch 491/500
4/4 [==============================] - 0s 7ms/step - loss: 6.0580e-07 - accuracy: 1.0000
Epoch 492/500
4/4 [==============================] - 0s 8ms/step - loss: 6.0472e-07 - accuracy: 1.0000
Epoch 493/500
4/4 [==============================] - 0s 8ms/step - loss: 6.0183e-07 - accuracy: 1.0000
Epoch 494/500
4/4 [==============================] - 0s 8ms/step - loss: 5.9388e-07 - a

In [13]:
# 9. 모델 사용
test = [['cold', 'PP'],
        ['down', 'PP'],
        ['desk', 'PP'],
        ['love', 'PP'],
        ['olve', 'PP'],
        ['lvoe', 'PP'],
        ['loev', 'PP'],
        ['eovl', 'PP']]
enc_in, dec_in, dec_out = encoding(test)
enc_in = np.array(enc_in)
dec_in = np.array(dec_in)
enc_in.shape, dec_in.shape

((8, 4, 171), (8, 3, 171))

In [14]:
pred = model.predict([enc_in, dec_in])
pred

1/1 [==============================] - 1s 639ms/step


array([[[1.47158015e-14, 9.99760991e-19, 1.01405531e-14, ...,
         1.44566623e-13, 9.30488217e-13, 1.69603595e-07],
        [1.55518436e-15, 4.50136177e-08, 1.69049523e-15, ...,
         2.27538734e-11, 6.36702554e-15, 1.00974000e-10],
        [3.64209444e-18, 9.99999881e-01, 5.43770833e-18, ...,
         4.96591321e-13, 2.25626644e-18, 3.34785707e-15]],

       [[6.13802019e-14, 3.55031192e-15, 4.48322579e-14, ...,
         2.32085315e-12, 8.64551052e-12, 9.82402426e-08],
        [1.43639495e-13, 3.37156827e-08, 1.65119722e-13, ...,
         2.82351653e-08, 3.45948042e-14, 1.60026659e-10],
        [5.51078206e-16, 1.00000000e+00, 1.00004966e-15, ...,
         8.42830528e-11, 3.91015819e-17, 3.70857757e-14]],

       [[1.78460423e-13, 3.52044151e-21, 1.26858661e-13, ...,
         7.56495561e-14, 4.96854469e-08, 2.48354365e-10],
        [2.45575858e-13, 4.06364612e-13, 1.47612093e-13, ...,
         3.15981158e-10, 1.36333805e-10, 7.64764779e-11],
        [1.42144411e-14, 9.99999642e

In [15]:
pred.shape

(8, 3, 171)

In [16]:
# alpha[32], char_to_num['감']
# 출력형식 : cold -> 감기(32, xx, 1)
for i in range(len(pred)):
    eng = test[i][0]
    hat = pred[i].argmax(axis=1)
    kor = alpha[hat[0]] + alpha[hat[1]]
    print("{} -> {}({})".format(eng, kor, hat))

cold -> 감기([32 46  1])
down -> 아래([111  67   1])
desk -> 책상([149  97   1])
love -> 사랑([96 66  1])
olve -> 사랑([96 66  1])
lvoe -> 사랑([96 66  1])
loev -> 사랑([96 66  1])
eovl -> 도다([62 60  1])
